# Django - Wprowadzenie
*[Mikołaj Leszczuk](mailto:mikolaj.leszczuk@agh.edu.pl), [Agnieszka Rudnicka](mailto:rudnicka@agh.edu.pl)*

* Po co nam baza danych?
* "Klasyczne" zarządzanie bazami danych
* Tworzenie tabeli w Django - podgląd migracji
* Aplikowanie migracji
* Przeglądanie bazy danych przy pomocy DB Browser
* Podsumowanie
* Panel administracyjny

## Po co nam baza danych?

Najprościej mówiąc - to kontener w którym zapisane będą informacje o użytkownikach naszej strony, książki, aktualności i wszystkie inne rzeczy które dodamy. Baza danych w naszym przypadku pozwala na przechowanie danych, które będą dodawane przez użytkownika aplikacji, nie przez programistę. My jako programiści jedynie ustalimy reguły wyznaczając modele. Na tych zajęciach skipimy się na bazach relacyjnych, takich jak sqlite3. Warto jednak wiedzieć, że typów baz danych jest wiele, jednym z takich, który znamy z codzienności jest arkusz kalkulacyjny.

W tym konspekcie zaczniemy od modelu przeznaczonego do przechowywania książek.

Zobaczmy jak to wygląda w praktyce...

## "Klasyczne" zarządzanie bazami danych

Gdybyśmy nie mieli narzędzi takich jak Django i inne narzędzia, praca z bazami danych (w większości) wymagałaby nauczenia się dodatkowego języka: SQL.

Przykładowe tworzenie tabeli z wykorzystaniem SQL:

```sql
CREATE TABLE "movies_movie" (
    "id" integer NOT NULL PRIMARY KEY AUTOINCREMENT,
    "title" varchar(100) NOT NULL,
    "short_description" text NOT NULL,
    "published_at" datetime NOT NULL);
COMMIT;
```

Byłby to kod, który trzeba przetestować i gdzieś zapisać. A następnie pamiętać, żeby wykonać na odpowiedniej bazie danych. Gdyby w trakcie projektu wygląd tej tabeli miałby się ulec zmianie (a często się tak dzieje!) wymagane byłyby kolejne skrypty napisane w SQL. A to tylko początek problemów...

## Tworzenie tabeli w Django - podgląd migracji

Tworzenie tabeli z wykorzystaniem Django (w pliku [`models.py`](http://localhost:8888/edit/Documents/Courses/Django/movies/models.py)):

```python
class Movie(models.Model):
    title = models.CharField(max_length=100)
    short_description = models.TextField()
    published_at = models.DateTimeField()
```

Poniższym poleceniem możemy wygenerować migrację. Migracja w tym przypadku to skrypt w Pythonie, który pozwala na utworzenie potrzebnych tabel w bazie danych, która jest podpięta do aplikacji. W naszym przypadku tabele zostaną stworzone w bazie `sb.sqlite3`, która znajduje się domyślnie w głównym katalogu całego projektu/repozytorium kodu.

In [ ]:
!python3 manage.py makemigrations

Po wykonaniu powyższego polecenia w katalogu `migrations` pojawi się nowy plik nazwany `0001_initial.py`. Migracje są automatycznie numerowane, natomiast ich nazwę można ustawić (`python manage.py makemigrations -n nazwa` wygeneruje `numer_nazwa.py`).

Na pierwszy rzut oka może się wydawać, że kodu/pracy wcale nie jest mniej. Jednak kluczowe zyski z korzystania z frameworka to:

* nie trzeba się uczyć dodatkowego języka (SQL)

* framework jest dobrze przetestowany przez twórców i użytkowników, więc możemy założyć, że działa

* przy każdej zmianie w modelu wystarczy uruchomić polecenie w konsoli, a SQL zostanie wygenerowany automatycznie - nie musimy się zastanawiać co się zmieniło

Innymi słowy - oszczędzamy czas dzięki gotowym narzędziom :)

Podgląd migracji

Jeśli chcielibyśmy jedynie podejrzeć co zostanie wygenerowane możemy wykorzystać dodatkowe flagi:

```sh
--dry-run -v 3
```

Pierwsza część pozwala na "wykonanie na sucho", czyli cała maszyneria zostanie uruchomiona, ale migracja nie zostanie zapisana na dysku/stworzona. `-v 3` natomiast pozwala na wypisanie wszystkich szczegółów procesu generowania migracji a także samej migracji.

 Wynik działania tego polecenia możemy zobaczyć poniżej:

```sh
python3 manage.py makemigrations --dry-run -v 3

# Powyższe polecenie wypisze na konsole poniższe informacje

Migrations for 'movies':
  movies\migrations\0001_initial.py
    - Create model Movie
Full migrations file '0001_initial.py':
# Generated by Django

from django.db import migrations, models

class Migration(migrations.Migration):

    initial = True
    
    dependencies = [
    ]
    
    operations = [
        migrations.CreateModel(
            name='Movie',
            fields=[
                ('id', models.AutoField(auto_created=True, primary_key=True, serialize=False, verbose_name='ID')),
                ('title', models.CharField(max_length=100)),
                ('short_description', models.TextField()),
                ('published_at', models.DateTimeField()),
            ],
        ),
    ]
```

Jak widzimy, najpierw narzędzie poinformowało nas, że tworzona zostaje migracja dla aplikacji "movies" a następnie w terminalu wypisana została cała migracja (ten krótki kawałek kodu w Pythonie począwszy od `# Generated by Django` do samego końca).

## Aplikowanie migracji

Teraz kiedy mamy już napisany podstawowy model oraz wygenerowaną migrację trzeba ją zaaplikować na bazę danych.

Utworzy to odpowiednią tabele na nasze książki w bazie danych.

In [ ]:
!python3 manage.py migrate

W outpucie, w linijce `Applying movies.0001_initial... OK` widnieje nasza migracja! Ona została zaaplikowana.

## Przeglądanie bazy danych przy pomocy DB Browser

Do przeglądania bazy danych SQLite można wykorzystać [https://sqlitebrowser.org/](https://sqlitebrowser.org/)

Jest to mały darmowy program, który uruchommy.

## Podsumowanie

Reasumując, aby przechowywać informacje w bazie danych musimy:

1. Opisać w Django (plik `models.py`) jak ma się nazywać model, jakie ma mieć pola oraz jakie właściwości mają te pola posiadać. (np. pole "data_utworzenia", typ: data, wymagane)
2. Wygenerować migrację (polecenie: `python manage.py makemigrations`)
3. Zaaplikować migrację na bazie danych, co spowoduje utworzenie odpowiednich tabel lub wprowadzenie zmian w istniejących. (polecenie: `manage.py migrate`)

## Panel administracyjny

Najszybszym (i dla niektórych najwygodniejszym) sposobem, żeby dodać nowe książki będzie wykorzystanie panelu administracyjnego Django. Jest to narzędzie dostarczone z frameworkiem, domyślnie włączone.

Aby z niego skorzystać musimy zarejestrować nasz model `Movie` jako obsługiwany przez panel administracyjny. W tym celu otwórzmy plik [`admin.py`](http://localhost:8888/edit/Documents/Courses/Django/movies/admin.py) znajdujący się w katalog aplikacji ([`movies/admin.py`](http://localhost:8888/edit/Documents/Courses/Django/movies/admin.py)).